# Clustering text documents using k-means 



In [ ]:
import numpy as np

from sklearn.datasets import fetch_20newsgroups



In [ ]:
categories = [
    "alt.atheism",
    "talk.religion.misc",
    "comp.graphics",
    "sci.space",
]

dataset = fetch_20newsgroups(
    remove=("headers", "footers", "quotes"),
    subset="all",
    categories=categories,
    shuffle=True,
    random_state=42,
)



In [ ]:
labels = dataset.target
unique_labels, category_sizes = np.unique(labels, return_counts=True)
true_k = unique_labels.shape[0]
print(f"{len(dataset.data)} documents - {true_k} categories")

>[!IMPORTANT]
>This exercise is based on the following scikit-learn example: [Clustering text documents using k-means](https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#sphx-glr-auto-examples-text-plot-document-clustering-py)

>[!NOTE]
> License: BSD 3 clause
> Author: Peter Prettenhofer <peter.prettenhofer@gmail.com>
> Lars Buitinck
> Olivier Grisel <olivier.grisel@ensta.org>
> Arturo Amor <david-arturo.amor-quiroz@inria.fr>